<a href="https://colab.research.google.com/github/Malik-Raheel/Trading-Strategies/blob/main/Shortlist_Asset_with_ATR_OBV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tracking Assets from Coingeco: Positive OBV and ATR Assets from top 500 Assets listed at Coin Gecko. Trading Assets that are showing good Strength recently. High volatility and Showing Momentum.
It will extract valuable Assets. Shortlist your asset for trading.

In [ ]:
# Install required libraries
!pip install pandas numpy ta requests pycoingecko

In [ ]:


import pandas as pd
import numpy as np
from ta.volume import OnBalanceVolumeIndicator
from ta.volatility import AverageTrueRange
from pycoingecko import CoinGeckoAPI
import time

# Initialize CoinGecko API client
cg = CoinGeckoAPI()

# Function to fetch top 500 assets from CoinGecko
def fetch_top_500_assets():
    assets = []
    for page in range(1, 6):  # 5 pages x 100 assets = 500 assets
        data = cg.get_coins_markets(vs_currency="usd", order="market_cap_desc", per_page=100, page=page)
        for asset in data:
            assets.append({
                "id": asset["id"],
                "symbol": asset["symbol"],
                "name": asset["name"]
            })
        time.sleep(1)  # Respect API rate limits
    return pd.DataFrame(assets)

# Function to fetch historical data for an asset
def fetch_historical_data(asset_id, days="30", interval="daily"):
    data = cg.get_coin_market_chart_by_id(id=asset_id, vs_currency="usd", days=days, interval=interval)
    prices = data["prices"]
    volumes = data["total_volumes"]
    df = pd.DataFrame({
        "time": [p[0] for p in prices],
        "price": [p[1] for p in prices],
        "volume": [v[1] for v in volumes]
    })
    df["time"] = pd.to_datetime(df["time"], unit="ms")
    return df

# Function to calculate OBV and ATR
def calculate_indicators(df):
    obv = OnBalanceVolumeIndicator(close=df["price"], volume=df["volume"])
    atr = AverageTrueRange(high=df["price"], low=df["price"], close=df["price"])
    df["OBV"] = obv.on_balance_volume()
    df["ATR"] = atr.average_true_range()
    return df

# Main function to track trending assets
def track_trending_assets():
    trending_assets = []
    top_assets = fetch_top_500_assets()
    print(f"Fetched {len(top_assets)} assets from CoinGecko.")

    for _, asset in top_assets.iterrows():
        try:
            # Fetch historical data
            df = fetch_historical_data(asset["id"])
            df = calculate_indicators(df)

            # Check for increasing OBV
            if len(df) > 1 and df["OBV"].iloc[-1] > df["OBV"].iloc[-2]:
                # Check for good ATR
                if df["ATR"].iloc[-1] > df["ATR"].mean():  # Compare to average ATR
                    trending_assets.append({
                        "id": asset["id"],
                        "symbol": asset["symbol"],
                        "name": asset["name"],
                        "last_price": df["price"].iloc[-1],
                        "last_OBV": df["OBV"].iloc[-1],
                        "last_ATR": df["ATR"].iloc[-1]
                    })
        except Exception as e:
            print(f"Error processing {asset['id']}: {e}")
        time.sleep(0.2)  # Respect API rate limits

    return pd.DataFrame(trending_assets)

# Execute the tracker
trending_assets = track_trending_assets()
print("Trending Assets with Increasing OBV and Good ATR:")
print(trending_assets)

# Save results to a CSV file
trending_assets.to_csv("trending_assets.csv", index=False)
print("Results saved to 'trending_assets.csv'.")
